<a href="https://colab.research.google.com/github/RolandoOrtizB/Bootcamp/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'proyecto-3-segmentacin-de-clientes-en-ecommerce:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1866450%2F3167330%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240521%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240521T015304Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1793ad2d9fd58e2470dcf223774c6017bed512ddb1eb0600c12abb97e791b7ce2f9095987625d27e8c8383eac8201e559d2450befa3be310505de34063f845798a7fee21004c483a741a1b8e3e0e7a07e8769d5a418a968bd6006b758531aaad3e6190eea37b177bcbd928a521017dbcfce7a106ccd5da87fd1c96d4302e1f9ba001a65525617ab602889a5f10d2cbb40f48337584caf66d185fc16a8af0e62699b5a3c7a8de87c48a42c67adf7e73c2ed9a2757b8acaaae0793d5a722cb6be78d193e4656a9ff760dd93be89c1393ad3170a8ccaa61836dc789519f44e8abc76743c6f528d8d4159448ce19453573d17dddfbd140c1adc9de4f67da23f0ce74'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# 1. Contexto

En este proyecto simularás ser parte del equipo de una consultora estratégica enfocada en Business Intelligence. Utilizando datos históricos de una empresa de ventas al por mayor deberás hacer un análisis completo que incluya:

- **Pre-procesamiento de los datos:** revisar la calidad de la base de datos, eliminar datos duplicados que no tengan sentido, encontrar datos faltantes y reemplazarlos/eliminarlos, y buscar datos que no correspondan (por ej números negativos en columnas que generalmente son números positivos). El pre-procesamiento de los datos es un paso fundamental y básico de todo análisis de datos.
- **Análisis exploratorio**: utilizando una herramienta de visualización como Google Data Studio deberás graficar, resumir y estructurar la información más relevante para poder. La idea es poder mostrar visualmente las características principales del dataset, con un enfoque en métricas de venta como las que enlista tu jefe en el brief.
- **Análisis por cohorte**: esta herramienta ya la conoces. Deberás aplicar un análisis por cohorte para los clientes de UK y para los extranjeros para ver cuál de los dos públicos son clientes más leales.
- **Segmentación de clientes**: finalmente, tendrás que hacer una segmentación de los clientes basándote en tres características: Recencia, Frecuencia y Monto.

Es un análisis bien completo que te permitirá hacer algunas recomendaciones a UK Merch para que aprovechen de mejor manera sus recursos.

> *“Las empresas capaces de utilizar los datos de los clientes para comprender mejor cómo piensan, sienten y toman decisiones de compra, tienen una ventaja.”*

¡Vamos por el dataset! 👩‍💻

---

# 2. Importar, entender el dataset y limpiar los datos

Para este proyecto contamos con un archivo .csv en el [dataset de Kaggle](https://www.kaggle.com/datacertlaboratoria/proyecto-3-segmentacin-de-clientes-en-ecommerce). Antes de importarlo a Google Sheets te recomiendo que aproveches el visor de datos (Data Explorer) de Kaggle para revisar a modo general la estructura de los datos.

![image.png](https://drive.google.com/uc?export=view&id=1zd2u52tyhgOLSB4NEgHBKrcAs6jwq-64)

Al importarlo en Google Sheet y mirar la primera fila podemos interpretar ese registro de la siguiente forma: el cliente 15528 de United Kingdom hizo una compra el 30 de marzo del 2021 a las 4:14PM por un monto de 229.33 libras esterlinas que incluye 123 productos, y que generó la factura cuyo número es el 548370.

> **⚠️ Importante**: si al importar los datos la columna de monto no te aparece con formato como número sino texto, prueba yendo a Archivo > Configuración y cambiando la configuración regional a Estados Unidos. El problema es que en la configuración latinoamericana muchas veces se confunde entre punto y coma como separador de miles o decimales. Al hacer el cambio y luego poniendo el formato de los datos como número o moneda debería solucionarse el problema.

Ahora que tenemos nuestros datos importados y entendemos a modo general la estructura de los datos comenzamos con la limpieza (o pre-procesamiento) de los datos.

### 2.1 Detección de nulos o vacíos

Después de revisar con mayor detalle las variables que se tiene en el dataset, es posible que este contenga ciertos registros vacíos o nulos, estos datos podrían ensuciar tu análisis. Para eso te recomendamos seleccionar cada columna y hacer uso de la opción **“Filtros”** en cada una de ellas, dejando solo los valores vacíos (o Blanks).

![image.png](https://drive.google.com/uc?export=view&id=1FqucNEw0BD2WPIF0PvtmbZnuwqz9vr_c)

> 👩‍💻 Una manera rápida de diagnosticar si el dataset tiene valores vacíos es usar la función “=CONTAR.BLANCO()” o "=COUNTBLANK()". Esta función recorre cada valor del rango y cuenta la cantidad de registros vacíos que encuentra.

Revisa columna por columna con la función de filtro o aplica la fórmula que cuenta los blancos en cada columna para encontrar valores vacíos. Luego reflexiona: esos valores vacíos que encontraste, ¿afectarán a tu análisis? ¿es mejor dejarlos o borrarlos? Esta decisión depende de cada situación. Si para este proyecto es de suma importancia que esos valores no estén vacíos (porque de otra forma ensuciarán el resultado) entonces puedes borrar esos registros (toda la fila). Si no afectan el análisis, los puedes dejar.

### 2.2 Detección y eliminación de duplicados

Algunas veces los usuarios cometen errores al registrar la información. Un error muy común es que un registro esté duplicado.

Hay que entender bien cuándo un dato duplicado es erróneo. Para nuestro caso sí encontramos más de una fila que tenga el mismo monto o la misma cantidad, no sería un error. Solo significaría que en momentos distintos, usuarios distintos han comprado la misma cantidad o han gastado el mismo monto. Eso es muy común que suceda.

Lo que no debería suceder en este caso es que tengamos dos veces la misma factura. Como se supone que cada factura tiene un valor único, entonces si encontramos otra fila con el mismo número de factura deberíamos borrar el duplicado.

Para eliminar duplicados puedes ocupar la función de Quitar duplicados (Datos > Borrado de datos > Quitar duplicados). A esta herramienta debemos decirle que columnas queremos que compare entre las filas. Como vimos anteriormente, queremos eliminar las filas que compartan valores en la columna de número de factura.

![image.png](https://drive.google.com/uc?export=view&id=1SLsfxeucLni6wHzQiHW8BpQIosFyFb-u)

### 2.3 Detección y eliminación de cantidades negativas

Si revisaste con atención el [Data Explorer de Kaggle](https://www.kaggle.com/datacertlaboratoria/proyecto-3-segmentacin-de-clientes-en-ecommerce) o si ya descubriste la función de Google Sheets "Estadísticas de Columna" (Datos > Estadísticas de Columna) te habrás dado cuenta que tanto en la columna de Monto como la de Cantidad hay valores negativos.

Estos valores negativos no necesariamente están incorrectos. De hecho, si revisaste con atención la descripción de la columna en el dataset de Kaggle te habrás dado cuenta que aquellas facturas que comienzan con la letra C corresponden a devoluciones y, por lo que podríamos asumir que esos valores negativos son por esa razón (una venta es positiva, una devolución negativa).

![image.png](https://drive.google.com/uc?export=view&id=1X9nKHZTY55BGIvoya5wxvT86K6x3W8C-)

Como solo queremos analizar las ventas del negocio (y no las devoluciones), deberíamos eliminar esos valores de nuestro dataset. Para hacerlo puedes usar filtros, en particular la opción **"Filtrar por condición"**, y luego eliminar las filas que tengan valores negativos.

> 👀 ¿Se te queda pegado el navegador? Esto es normal. Como estamos trabajando con una fuente de datos grande (más de 20.000 registros) es común que esto suceda. Más adelante en la certificación aprenderemos herramientas para manejar grandes volúmenes de datos en un instante.

¿Se te ocurren algunas otras validaciones que podríamos hacer con este dataset? Coméntalo con tus compañeras en Slack.

# 3. Análisis de las métricas de ventas

¡Enhorabuena! Ya tienes limpio el dataset, es momento de empezar a generar información para la compañía. La exploración de datos implica el uso de herramientas de visualización de datos y técnicas estadísticas para identificar las características de un conjunto de variables.

Primero comenzaremos revisando la cantidad de facturas que genera cada país.

### 3.1 Análisis de número de facturas por país

Algo interesante de analizar es la cantidad de facturas que emite cada país y el porcentaje de las facturas que cada uno representa.

![facturas.png](https://drive.google.com/uc?export=view&id=1kE8_jLtjFQldV9u6GYEzZseLr_YT3Mfv)

Intenta generar esta información utilizando [tablas dinámicas](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/18339974#search), y resumiendo la cantidad de facturas según cantidad y según porcentaje del total.

> 🤓 **Análisis** 🤓
>
> Es evidente que los clientes de UK son aquellos que realizan la mayor cantidad de compras en el supermercado, dado que el negocio nació en ese país. Los siguientes 5 países con mayor cantidad de compras son Alemania, Francia,  Irlanda, Bélgica y Holanda.
>
> ¿Por qué crees que estos países Europeos son los que más compran en UK Merch?

### 3.2 Análisis del monto total por país

Podemos hacer algo similar pero ahora tomando en cuenta la variable monto. Aquí podemos aprovechar de hacer un análisis un poco más profundo. Además del monto sumado y el porcentaje que cada país representa del total, sería interesante conocer el monto promedio, el monto mínimo y el monto máximo en cada caso.

![monto.png](https://drive.google.com/uc?export=view&id=193JbFo5TAuzEOjrD7on2Tps3NP3OMD-u)

En esta ocasión se puede observar que UK representa el 82% del monto de las ventas, sin embargo tenía casi el 90% de las transacciones. Esto quiere decir que existen países que en promedio gastan más por transacción en UK Merch. Esto se puede validar en la columna de “Monto Promedio” donde vemos que UK tiene uno de los promedios más bajos por transacción, mientras que Holanda y Australia tienen los promedios más altos.

Otro tema importante y que puede causar ruido es que tenemos transacciones cuyo monto es 0. Este es un punto que vale la pena levantar al cliente para entender más el contexto. Puede ser un error de digitación o puede ser que en algunos casos regalen la mercancía (quizás sin estar al tanto de ello).

> 👩‍💻 Puedes probar con otros campos calculados como la [mediana](https://ibmcsr.udemy.com/course/estadistica-para-no-estadisticos/learn/lecture/27411654#overview) (MEDIAN) o la [desviación estándar](https://ibmcsr.udemy.com/course/estadistica-para-no-estadisticos/learn/lecture/27411668#overview) (STDEV) y complementar tus conclusiones.

Una buena idea sería hacer este mismo análisis pero ahora para la cantidad de productos de cada factura 🤓.

### 3.3 Facturas generadas por mes

¡Bravo! A partir de este primer análisis has encontrado insights (hallazgos) importantes y sobre todo has conocido más sobre el negocio. Una buena idea para complementar un análisis exploratorio de un dataset es agregando gráficos. Estos son muy útiles para identificar diferentes patrones de datos de una forma visual, o vislumbrar comportamientos atípicos o estacionales.

Para realizar análisis por mes, tendremos que hacer un procesamiento a la columna “Fecha de factura”. En esa columna se aloja la fecha de la factura pero con mucha información que no necesitamos (por ejemplo el día, la hora y los minutos). Para esto tenemos que crear una nueva columna "AÑO - MES" en donde nos gustaría que apareciera la fecha de la forma 2021-03 para cada fila.

La fórmula que nos puede ayudar a lograr esto es TEXTO. Investiga en [la documentación de Google Sheets](https://support.google.com/docs/answer/3094139?hl=es-419) o investiga en Google para llegar al resultado esperado.

![mesano.png](https://drive.google.com/uc?export=view&id=1V8j4E6szJsb6OyVmxkWpTyvQp2UKJ6AX)

Con esta nueva columna ya podemos generar una tabla dinámica que nos cuente la cantidad de facturas en cada mes, para luego graficar esos valores:

![mesano.png](https://drive.google.com/uc?export=view&id=1xzl9eyZCybZSU81yZMawxab8GGigVuZs)

> 🤓 **Análisis** 🤓
>
> No podemos asegurar que hay una estacionalidad dado que sólo tenemos un año de historia de datos. Pero algo que se aprecia es que en noviembre hay un crecimiento grande en las ventas, tal vez esto pueda deberse a las ventas pre fiestas navideñas, sin embargo vemos que existe una caída pronunciada en el mes de diciembre del 2021 😱.

> ¿Por qué crees que se da esta caída? Revisa las fechas de las compras de diciembre del 2021 y tendrás tu respuesta 🧙‍♀️.

### 3.4 Número de facturas UK vs extranjeros por mes

Dado que UK Merch abrió sus fronteras a otros países, sería una buena idea reportar como están yendo estas ventas y poder evidenciar si existe el mercado. Por lo que vamos a segmentar nuestros clientes en dos grupos: aquellos que pertenecen a UK y aquellos que no pertenecen a UK.

Para esto creamos una columna "¿Pertenece a UK?" y ocupamos la fórmula:

```SQL
=SI(PAIS="United Kingdom";"SI";"NO")
```

Lo que hace esta fórmula es evaluar la condición (primer argumento). Si esa condición es verdadera, entonces nos retorna el segundo argumento, si es falsa nos retorna el tercer argumento.

Por lo tanto, si en esta fórmula el país que estamos comparando es "United Kingdom" nos devuelve en la celda "SI", y "NO" en caso contrario.

![mesano.png](https://drive.google.com/uc?export=view&id=1qBzNb8lJwp2ZEiZiSyLc5YNx0eWfV-KF)

Con esta nueva variable podemos crear una tabla dinámica que cuente la cantidad de facturas para cada uno de los casos de la nueva columna "¿Pertenece a UK?". A diferencia de las otras tablas dinámicas que hemos armado, esta necesita que le digamos qué valores debe contar en las columnas:

![mesano.png](https://drive.google.com/uc?export=view&id=1EaKBq5x1JaYJZMopDxMHyGyi8Rk-wSIM)

Con esta nueva tabla dinámica podemos generar una serie temporal que compare a ambos tipos de clientes:

![mesano.png](https://drive.google.com/uc?export=view&id=15uulXHjg2yyHxDgQBrOA4JUntuD4xWbC)

> 🤓 **Análisis** 🤓
>
> Sin tomar en en cuenta el mes atípico de diciembre 2021, podemos ver que ambos segmentos van creciendo.
>
> ¿Qué conclusión puedes sacar con respecto al mercado de clientes no-UK?

### 3.5 Cantidad de clientes nuevos por mes

Ya vimos que Noviembre fue un mes muy bueno para UK Merch. Se generaron muchas más facturas. Este aumento en venta se puede deber a que algunos pocos clientes  hicieron muchas compras, o que una gran cantidad de clientes hicieron aumentar el volumen de compras. Para resolver esta duda, una buena idea es graficar mes a mes la cantidad de clientes únicos y ver si el aumento de las facturas está relacionado con el aumento de clientes únicos.

Nuevamente, puedes hacer una tabla dinámica que use como filas los meses-años y que el valor que calcule sea la cantidad de clientes. Esto se puede calcular ocupando la columna ID Cliente y haciendo un recuento único (COUNTUNIQUE) para que solo tome valores únicos.

> 🤔 ¿Qué pasa si no ocupamos esta función y hacemos un recuento simple (COUNTA)?

![mesano.png](https://drive.google.com/uc?export=view&id=1rlaCLc628hF56EV3aPd5Gn6JuvfwzVcC)

Esta nueva tabla dinámica también la podemos graficar:

![mesano.png](https://drive.google.com/uc?export=view&id=17SPsWxTApxeSHaKvAW7DC2Qr2mAVpuPZ)

> 🤓 **Análisis** 🤓
>
> Viendo este nuevo gráfico, ¿el éxito de noviembre se debe a que unos pocos clientes hicieron muchas compras?, ¿o a que también creció el número de clientes que hicieron compras ese mes?

### 3.6 Agrega otros indicadores y ordena la información

¡Hasta el momento has generado tablas y gráficos muy útiles! Ellos te han permitido entender de mejor manera como se comportaron las ventas del negocio el año pasado. Pero hay muchas otras métricas que puedes revisar, ¡anímate a complementar tu reporte! Algunas ideas:

- Analizar el monto total de venta por mes
- Tabular los clientes que más compras han realizado
- Graficar los clientes que más han gastado
- Analizar cómo se comporta el monto mes a mes en clientes UK y no-UK
- Lo mismo pero también para la cantidad de productos
- ¡Agrega las que a ti te parezcan útiles!

Toda esa información valiosa probablemente está repartida en varias pestañas, tablas y gráficos. Ahora es una buena idea para comenzar a estructurarlos en un reporte o dashboard que sea ordenado y amigable para que alguien externo pueda comprender tus conclusiones.

> 🎦 En [la sección 3 del curso de Google Sheets](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/18081049#search) que está en el [learning path de Skillsbuild](https://skills.yourlearning.ibm.com/activity/PLAN-F2DC3A8C2759) puedes encontrar videos valiosos que te muestran [como estructurar un buen reporte en hojas de cálculo](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/18350484#search).

### 3.7 Desafío 🤯 (opcional)

Si quieres ir al siguiente nivel y aprender una herramienta muy útil en el proceso te desafiamos a que realices tu análisis y reporte en [Google Data Studio](https://datastudio.google.com/) 😱. Esta es una herramienta que te permite conectar diversas fuentes de datos y luego crear visualizaciones y reportes dinámicos. Este tipo de herramientas, llamadas visualizadores, son elementales en el quehacer de una Analista de Datos.

No te preocupes, no estarás sola. Te recomendamos mirar [la sección 5 del curso de Google Sheets](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/19671156#search) que está en [el learning path de Skillsbuild](https://skills.yourlearning.ibm.com/activity/PLAN-F2DC3A8C2759). Ahí aprenderás a [importar tus datos desde Google Sheets a Data Studio](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/19713220#search) y a [crear tus primeros gráficos](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/19716208#search) y [visualizaciones geográficas](https://ibmcsr.udemy.com/course/tablas-dinamicas-dashboards-macros-y-mas-con-google-sheets/learn/lecture/19838104#search).

# 4. Análisis de cohortes

Es momento de responder a la pregunta ¿Qué tan fieles o comprometidos son los clientes de UK Merch?. Para responder esto haremos uso de una técnica que ya aplicaste en tu [proyecto de retención de startup](https://www.kaggle.com/datacertlaboratoria/proyecto-2-startup-tecnolgica), el **análisis por cohortes**. Según la definición, para aplicar este análisis es necesario separar a los clientes de acuerdo a su fecha de ingreso a nuestro servicio. A diferencia del proyecto anterior, donde estaba claro cuál era la fecha de registro ya que era la fecha en que el cliente comenzó a pagar la suscripción, aquí la situación es distinta. Una alternativa es clasificar a los clientes de acuerdo al mes de su primera compra. ¡Vamos por ello 🙌!

### 4.1 Replicar la data en otra pestaña y crear la columna cohorte

Necesitamos copiar la data en otra pestaña. Para eso podemos ocupar la ya conocida función [Query](https://juansguzman.com/introduccion-a-la-funcion-query-en-google-sheets/).

A la misma data le agregamos una columna extra que llamaremos "Cohorte". Aquí queremos almacenar el valor para cada cliente de su fecha de ingreso. Tal como vimos anteriormente, esta fecha de ingreso al cohort será su **primera fecha de compra**.

Para encontrar este valor ocuparemos la función BUSCARV sobre el dataset original.

```SQL
=BUSCARV(IDCLIENTE;DATASET;COLUMNA DE MES-AÑO;0)
```

Esta fórmula buscará la primera ocurrencia del ID del cliente en el dataset y retornará la fecha asociada. ¿Cómo nos aseguramos que la fecha que nos retorne la fórmula sea la primera compra del cliente? Muy simple 🤓, podemos ordenar el dataset donde buscaremos según su AÑO-MES de forma **ascendente**, y así la primera ocurrencia que encuentre nuestra fórmula BUSCARV será la primera fecha de compra del cliente.

![imagen.png](https://drive.google.com/uc?export=view&id=1wqlD2OGfC_1BMtzsxSkVdyWCpM7uikab)

Con esta tabla ordenada, en nuestra nueva pestaña podemos ocupar la fórmula BUSCARV para extraer el cohort de cada cliente.

![imagen.png](https://drive.google.com/uc?export=view&id=160IRC_5GnzHzA39aVq8FoeICJZs12m1T)

### 4.2 Crear tabla dinámica

Al igual que en nuestro proyecto 2, vamos a crear una tabla dinámica cuyas filas sean el cohorte, las columnas sean las fechas año-mes y los valores sean la cantidad de **clientes únicos** que hicieron compras esos meses.

¿Por qué clientes únicos? Porque lo que nos interesa saber es si un cliente hizo una compra en ese mes. No nos interesa saber si hizo varias compras, con que ya haya hecho una compra es suficiente para decir que todavía está "engaged" o "retenido" en esa fecha.

![imagen.png](https://drive.google.com/uc?export=view&id=1svFFnws83Pwx5YBFFHchL8etqk3EGFn6)

> 🤓 **Análisis** 🤓
>
> Ya sabemos leer este tipo de tablas. Por ejemplo, en el cohorte de los clientes que comenzaron a comprar en UK Merch en febrero del 2021 (fila 3), de los 380 que comenzaron comprando ese mes, solo 71 siguen comprando dos meses después.
>
> Hay una baja sustancial en la última columna (el último mes), ¿a qué se debe esto? Recuerda que el mes de diciembre final tiene un tratamiento especial.

### 4.3 Transformar tabla dinámica a análisis por mes

Por nuestra experiencia en el proyecto anterior, sabemos que estos datos tenemos que modificarlos. Primero tenemos que mover todos los cohorts para que comiencen en la primera columna. Para esto sabemos ocupar la función QUERY o una combinación de BUSCARV+FILAS+COLUMNAS (o si prefieres, simplemente mover manualmente la información 🤫), para luego renombrar las columnas como Mes 1, Mes 2, y así.

Luego de eso tenemos que dividir los números por los clientes que comenzaron, para calcular el porcentaje de ellos que continúa en cada mes.

Finalmente, ocupar el formato condicional para darle un aspecto de mapa de calor (semáforo) para poder ver gráficamente donde están las fugas y las retenciones.

> **⚠️ Importante**: una vez hecho este análisis, vale la pena replicar el proceso para conocer la retención de los clientes UK y los clientes no-UK. Esto lo puedes hacer aplicando filtros en tu tabla dinámica o filtrando tu información en el dataset copiado. Una vez que los tengas puedes agregar los insights que recojas en tu presentación final.
>
> Cuando analices los cohorts asegurate de comparar 🍐 con 🍐 y 🍎 con 🍎. Me refiero a que los porcentajes de retención que obtengas no son comparables con los que obtuviste en tu proyecto de startup. Son negocios e industrias distintas. Más bien vale la pena investigar online cuánto es una retención de clientes "sana" en el contexto de la venta online.

# 5. Segmentación de clientes

Primero lo primero. En el enunciado de proyecto habrás leído que tu jefe recomendó hacer una segmentación por Recencia, Frecuencia y Monto, o RFM por sus siglas. ¿Qué es esto 😕?

Esta metodología permite clasificar a los clientes en las tres variables mencionadas:

![rfm.png](https://drive.google.com/uc?export=view&id=1I3BJWMOiUjoaMbTuHUBnsLeuC01tO6oV)

- **Recencia**: Los clientes que han comprado recientemente tienen mayor tendencia a adquirir nuevos productos que aquellos que tienen tiempo sin hacer ninguna compra. Esta variable mide el tiempo que ha transcurrido desde la última compra. Para calcularla es necesario identificar la última compra que realizó el cliente y restarlo de la fecha de hoy. Esta puede estar en días, meses, etc.
- **Frecuencia**: Los clientes que han comprado repetidas veces al ser comparados con aquellos que no, son más propensos a seguir haciendo compras. Está variable se puede calcular como el número de visitas del cliente, número de meses distintos que visitó el cliente, número de productos que compró.
- **Monto**: Los clientes que en el pasado han sumado más dinero en todas sus compras, son mayoritariamente más propensos a seguir generando ingresos a una empresa. Para calcular esta variable, básicamente tendríamos que sumar todo el gasto del cliente en el periodo de análisis.

Una vez que tenemos calculadas las 3 variables para todos los clientes, los categorizamos de acuerdo al **cuartil** en que se encuentra cada cliente para cada variable. Es importante notar que este análisis se puede hacer por cuartiles, como también por quintiles, tercios, percentiles, o la división que prefiera la analista de datos. Mientras más divisiones, más fino el análisis.

> 🤔 ¿Qué es un cuartil? Si ordenas tus datos de menor a mayor y los divides en cuatro grupos, obtienes cuartiles. Cuando decimos que un cliente pertenece al primer cuartil de monto, quiere decir que está en el primer grupo de cuatro de los que menos gasta. Si por ejemplo, otro cliente está en el cuartil 3 en monto, quiere decir que está en el tercer grupo que más gasta, por sobre el primer y el segundo.
>
> Si quieres aprender más sobre cuartiles, percentiles y como aplicarlos en hojas de cálculo te recomiendo [este video](https://ibmcsr.udemy.com/course/estadistica-para-no-estadisticos/learn/lecture/27411660#overview) del [curso estadística para no estadísticos](https://ibmcsr.udemy.com/course/estadistica-para-no-estadisticos/learn/lecture/27411570#overview) del [learning path de Skillsbuild](https://skills.yourlearning.ibm.com/activity/PLAN-F2DC3A8C2759).

Entonces un buen cliente pertenece en recencia al cuartil 1 (ya que compró hace poco), en frecuencia al cuartil 4 (por qué compra muy frecuentemente) y en monto al cuartil 4 (por que es de los que más gasta). Para crear la codificación buscamos el cuartil de cada cliente en cada una de las variables y luego unimos esos valores en un "código" que resuma su perfil. Veamos un ejemplo:

![imagen.png](https://drive.google.com/uc?export=view&id=1YhYWyIGj36O6BTOgT1PjqxmeX5wr6ZSi)

Miremos algunos ejemplos:
- **El cliente 1 es un excelente cliente** ya que ha comprado hace poco (pertenece al primer cuartil en recencia, Recencia=1), ha comprado varias veces (pertenece al cuarto cuartil en frecuencia, Frecuencia=4) y ha gastado montos grandes en sus compras (pertenece al cuarto cuartil en monto, Monto=4). A ellos los codificamos como 144 (Recencia=1, Frecuencia=4, Monto=4)
- El cliente 4 es **un cliente que compró hace mucho tiempo** (cuarto cuartil en recencia, Recencia=4), pero ha hecho varias compras en su historia (cuarto cuartil Frecuencia, Frecuencia=4) y hace compras relativamente caras (tercer cuartil en monto, Monto=3). Si juntamos los números su codificación es 443.

Luego podemos ir creando distintas codificaciones con caracterizaciones de los clientes:
- **Los mejores clientes (RFM=144): ** Clientes altamente comprometidos que han comprado lo más reciente, con mayor frecuencia, y han generado la mayor cantidad de ingresos.
- **Los clientes leales (RFM=X4X):**  En este tipo de codificación la “X” hace referencia a que el cliente puede pertenecer a cualquier cuartil en Recencia, pertenece al cuartil 4 en Frecuencia y puede estar en cualquier cuartil en Monto. Esto quiere decir que el cliente codificado con 242 o 344 serán considerados dentro de la categoría “Más leales”. Estos clientes son los que compran más a menudo en UK Merch.
- **Los clientes que más pagan (RFM=XX4):** Los clientes que han hecho las compras más caras en UK Merch.
- **Los clientes fieles (RFM=X41, RFM=X42):** Son clientes que suelen volver a tu tienda pero no gastan mucho.
- **Nuevos clientes (RFM=11X):** Compradores por primera vez en UK Merch
- **Los durmientes (RFM=44X):** Grandes clientes del pasado que no han comprado en un tiempo.

> 👀 Según esta categorización es posible que **un cliente pueda pertenecer a más de un segmento**, en este caso se tendrá que priorizar a qué segmento asignarlo de acuerdo a la necesidad del negocio. Por otro lado, habrá clientes que no estén segmentados, podrías **hacer una refactorización en las categorías** (volver a pensar las categorías con más información) como por ejemplo tus clientes que más pagan pasarán a ser XX3 y XX4, de está forma agregarás a los clientes que pertenecen al cuartil 3 y 4 de la dimensión Monto.

¡Vamos por esa segmentación 🤘!

### 5.1 Preparar un nuevo dataset

Para mantener el orden, vamos a volver a crear una nueva pestaña donde copiaremos solo las columnas que necesitamos de nuestro dataset original. Para calcular la recencia necesitaremos las fechas, para la frecuencia la cantidad de facturas (y por ende, el número de factura) y para el monto claramente necesitamos el monto. Como todo esto va asociado al cliente, también necesitamos su ID.

![imagen.png](https://drive.google.com/uc?export=view&id=1e2VVxzJ2Gi-inZw6TtZ0aJY4SIbVULpX)

Con esta información crearemos una tabla dinámica en donde las filas sean los ID de todos los clientes y en las columnas podamos ver tres piezas de información importantes:

1. La última fecha de compra del cliente (aquí puedes ocupar el agregador SUM)
2. La cantidad de facturas que están asociadas a cada cliente
3. La suma total del monto gastado por el cliente

Te recomiendo volver a copiar esta información en otro lugar de la pestaña para poder agregarle columnas y hacer la categorización ahí (¡gracias función QUERY!)

![imagen.png](https://drive.google.com/uc?export=view&id=1DhVX1zt94NN0LtD1foBM0INnq_mBV9VI)

Como ves hay varias columnas que debemos comenzar a calcular. La primera es Días, en donde queremos calcular la cantidad de días que han pasado desde la última compra del cliente y hoy (el día que estamos haciendo el análisis). Para esto es útil la función [DIAS](https://support.google.com/docs/answer/9061296?hl=es-419) (o DAYS). Con esto vamos a poder ver en qué nivel de Recencia se encuentra cada cliente. Si ha comprado hace pocos días, probablemente lo categoricemos en el primer o segundo cuartil, pero si su última compra fue hace muchos días, puede que termine en el tercer o cuarto cuartil.

Antes de categorizar cada valor en su cuartil, necesitamos saber cuales son los rangos de los cuartiles. Para esto, crearemos una mini tabla en donde calcularemos los cuartiles para los días, frecuencia de facturas y monto total, utilizando la función [CUARTIL](https://support.google.com/docs/answer/3094041).

![imagen.png](https://drive.google.com/uc?export=view&id=17jwwFJz9NsKZQSxDbWJk2mMeVITV37en)

Estos valores que calculaste en la tabla son los límites entre cuartil y cuartil. Miremos un ejemplo:

![imagen.png](https://drive.google.com/uc?export=view&id=11zaZscd6G7r43PIG4tPMTVXt_P6vEZJg)

Supongamos que para Frecuencia obtuviste los valores de arriba. Esto quiere decir que los clientes que hayan comprado entre 0 y 1.185 veces estarán en el primer cuartil. Aquellos que han comprado entre 1.185 y 5.365 pertenecen al segundo cuartil, y así hasta llegar al 4to cuartil.

> 👀 Aún cuando en la figura pareciera que los cuartiles se superponen, no es el caso. Los límites que ves en la imagen son eso: límites.

Por lo tanto, los valores de ejemplo que vemos a la izquierda podemos categorizarlos según el cuartil al que pertenecen:
- 22.43 está entre 12.795 y 27.23 por lo tanto pertenece al 4to cuartil
- 6.27 está entre 5.365 y 12.795 por lo que pertenece al 3er cuartil
- 0.81 está entre 0 y 1.185 por lo que pertenece al 1er cuartil

Esta misma categorización tenemos que hacerla en nuestras columnas de Recencia, Frecuencia y Monto, categorizando los valores de Fecha Máxima, # Facturas y Monto respectivamente.

Veamos el ejemplo de la primera celda de Recencia:

![imagen.png](https://drive.google.com/uc?export=view&id=1-o9VX9R0GAtuYKUoDamB4OaxNggVrO_3)

Lo que queremos hacer es comparar el valor de la cantidad de días (AG2=367) con los rangos de los cuartiles de la tabla que acabamos de calcular y ver dónde entra. Para lograr esto queremos probar varias condiciones con la funcion [IFS](https://support.google.com/docs/answer/7014145?hl=en):

```SQL
=IFS(CONDICION_1;VALOR_1;CONDICION_2;VALOR_2...)
```

En esta fórmula el primer argumento es una condición, el segundo argumento es el valor a dejar en la celda si la primera condición se cumple. Si esto no se cumple, entonces prueba la segunda condición. Si esta se cumple deja el valor 2 en la celda, sino sigue con el resto de las condiciones.

Para ir probando en qué rango de cuartil se encuentra el valor en AG2 vamos haciendo comparaciones:

- **Condición 1 (AG2<=$AP$34;$AO$34):** Si AG2 es menor o igual a AP34 (que es el primer límite de rango del cuartil), entonces deja el valor de AO34 (que es el valor 1, para indicar que pertenece al primer cuartil).
- **Condición 2 (AG2<=$AP$35;$AO$35):** Si AG2 es menor o igual a AP35 (que es el segundo límite de rango del cuartil), y como ya sabemos que no pertenece al primer cuartil, entonces deja el valor de AO35 (que es el valor 2, para indicar que pertenece al segundo cuartil).
- **Condición 3 (AG2<=$AP$36;$AO$36):** Si AG2 es menor o igual a AP36 (que es el tercer límite de rango del cuartil), y como ya sabemos que no pertenece al segundo cuartil, entonces deja el valor de AO36 (que es el valor 3, para indicar que pertenece al tercer cuartil).
- **Condición 4 (AG2<=$AP$37;$AO$37):** Si AG2 es menor o igual a AP37 (que es el cuarto límite de rango del cuartil), y como ya sabemos que no pertenece al tercer cuartil, entonces deja el valor de AO37 (que es el valor 4, para indicar que pertenece al cuarto cuartil).

En el caso de AG2=367 no se cumple la primera condición (porque no es menor o igual que 59, no se cumple la segunda condición (porque no es menor o igual que 92), tampoco se cumple la tercera (porque no es menor o igual que 183.5), pero si se cumple la cuarta condición (si es menor o igual a 415), por lo que dejamos el valor de AO37=4. Es decir, ese valor pertenece al cuarto cuartil.

Esta categorización tenemos que hacerla para todos los clientes y en las tres columnas de RFM.

![imagen.png](https://drive.google.com/uc?export=view&id=1rsjMqD-Ydl_C_2BIAbk4rCZk5yd2JNNQ)

> 🤓 **Análisis** 🤓
>
> Veamos estos resultados. El cliente 12346 quedó categorizado como 4 en Recencia, 1 en Frecuencia y 4 en Monto. Y esto sí tiene sentido ya que el hizo su última compra hace mucho tiempo (367 días), no ha comprado muchas veces en UK Merch (solo tiene una factura a su nombre) y si ha gastado mucho (más de 77.000 libras esterlinas, lo que es una compra grande).
>
> ¿Puedes hacer un análisis similar con el cliente 12347?

Ahora que tenemos a todos los clientes categorizados en cuartiles para su Recencia, Frecuencia y Monto, podemos utilizar filtros para ir asignándoles categorías como las que definimos más arriba (y todas las que se te ocurran).

![imagen.png](https://drive.google.com/uc?export=view&id=19cz2-2ziOTyX3YAQ4i_mWkX0FdIjwDuk)

> 👩‍💻 Siempre hay más de una forma de resolver este tipo de problemas en el análisis de datos. Si te das cuenta la forma que te acabamos de explicar es bien manual. Implica hacer condiciones para cada intervalo de los cuartiles y luego ir categorizando a mano. ¿Qué pasa si en vez de cuartiles hubieras elegido hacer este ejercicio con deciles? Entonces necesitarias 10 condiciones en un función IFS 😱. Pero siempre Google Sheet tiene una forma más automática para hacerlo.
>
> En este caso podemos utilizar la fórmula RANGO.PERCENTIL.EXC que calcula la posición en percentil (de 1 a 100, pero entre 0 y 1. Por ejemplo, 0.83 es el percentil 83) del dato con respecto al conjunto de datos. Si ese valor lo multiplicas por 3 (para llevarlo a valores entre 0 y 3) y luego le sumas 1 (para llevarlo a valores entre 1 y 4, es decir, cuartiles) y luego redondeas el resultado, puedes obtener automáticamente su categorización
>
> ![imagen.png](https://drive.google.com/uc?export=view&id=1KNGpczbK4RNDnrQHVIKNssWb_UfJw25d)
>
> Luego, para no tener que categorizar utilizando filtros, podrías definir una serie de condiciones dentro de una función IFS
>
> ![imagen.png](https://drive.google.com/uc?export=view&id=16x_LKa5A8KIyd_w2esG-kxjWkfTyNwe6)

Cuando tengas los números, puedes resumir la información en una tabla dinámica y hacer recomendaciones específicas a UK Merch. Por ejemplo, si el número de clientes durmientes es muy alto, una buena estrategia de marketing sería contactarlos y motivarlos a volver a comprar (por medio de promociones por ejemplo) ya que sabemos que son clientes que compran muy seguido pero que hace tiempo no compran. Otro caso hipotético podría ser el lanzamiento de un nuevo producto. En este caso podría convenir apuntar las estrategias de marketing a tus clientes leales (los que compran más frecuentemente) para que asi tu nuevo producto agarre tracción en el mercado. [Este artículo](https://blog.elogia.net/rfm-recency-frecuency-y-money-qu%C3%A9-valor-tiene-nuestro-cliente) te puede dar algunas buenas ideas.

Otra estrategia inteligente sería agregar la variable de si pertenece o no a UK al análisis y ver como cambia la segmentación. Y de esa forma se pueden hacer recomendaciones de publicidad más enfocada en clientes de UK (con un lenguaje y modismos del país) si vemos que hay oportunidades ahí.

De esta forma UK Merch puede aplicar la ley de Pareto y enfocar sus esfuerzos para que sus mejores clientes le traigan la mayor cantidad de ganancias. Una campaña pequeña bien ejecutada a un segmento específico de clientes puede ser mucho más efectiva que una campaña publicitaria grande y cara sin un foco claro.